# Solutions-04

As we have seen, the standard Bayesian approach to model fitting involves sampling the posterior, usually via a variant of Markov Chain Monte Carlo (MCMC). Though there are many very sophisticated MCMC samplers out there, the most simple algorithm (Metropolis-Hastings) is rather straightforward to code.

Here we'll walk through creating our own Metropolis-Hastings sampler from scratch, in order to better understand exactly what is going on under the hood.

If you'd like to view one possible solution, take a look at the [Solutions-04](Solutions-04.ipynb) notebook (but again, try to make an honest effort at this before you peek!)

## Preliminaries

As usual, we start with some imports:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
# If this causes an error, you can comment it out.
import seaborn; seaborn.set()

## Metropolis-Hastings Procedure

Recall the Metropolis-Hastings procedure:

1. Define a posterior $p(\theta~|~D, I)$
2. Define a *proposal density* $p(\theta_{i + 1}~|~\theta_i)$, which must be a symmetric function, but otherwise is unconstrained (a Gaussian is the usual choice).
3. Choose a starting point $\theta_0$
4. Repeat the following:

   1. Given $\theta_i$, draw a new $\theta_{i + 1}$ from the proposal distribution
   
   2. Compute the *acceptance ratio*
      $$
      a = \frac{p(\theta_{i + 1}~|~D,I)}{p(\theta_i~|~D,I)}
      $$
   
   3. If $a \ge 1$, the proposal is more likely: accept the draw and add $\theta_{i + 1}$ to the chain.
   
   4. If $a < 1$, then accept the point with probability $a$: this can be done by drawing a uniform random number $r$ and checking if $a < r$. If the point is accepted, add $\theta_{i + 1}$ to the chain. If not, then add $\theta_i$ to the chain *again*.
   
The goal is to produce a "chain", i.e. a list of $\theta$ values, where each $\theta$ is a vector of parameters for your model.
Here we'll write a simple Metropolis-Hastings sampler in Python.

Note that the ``np.random.randn()`` function will be useful: it returns a pseudorandom value drawn from a standard normal distribution (i.e. mean of zero and variance of 1).

## The Data

We'll use data drawn from a straight line model

In [ ]:
def make_data(intercept, slope, N=20, dy=2, rseed=42):
    rand = np.random.RandomState(rseed)
    x = 100 * rand.rand(20)
    y = intercept + slope * x
    y += dy * rand.randn(20)
    return x, y, dy * np.ones_like(x)

theta_true = (2, 0.5)
x, y, dy = make_data(*theta_true)

## Exercise

Walk through all the following steps, filling-in the code along the way.

First plot the data to see what we're looking at (Use a ``plt.errorbar()`` plot with the provided data)

In [ ]:
plt.errorbar(x, y, dy, fmt='o');

We're going to fit a line to the data, as we've done through the lecture:

In [ ]:
def model(theta, x):
    # the `theta` argument is a list of parameter values, e.g., theta = [m, b] for a line
    return theta[0] + theta[1] * x

---

We'll start with the assumption that the data are independent and identically distributed so that the likelihood is simply a product of Gaussians (one big Gaussian). We'll also assume that the uncertainties reported are correct, and that there are no uncertainties on the `x` data. We need to define a function that will evaluate the (ln)likelihood of the data, given a particular choice of your model parameters. A good way to structure this function is as follows:

In [ ]:
def ln_likelihood(theta, x, y, dy):
    # we will pass the parameters (theta) to the model function
    # the other arguments are the data
    return -0.5 * np.sum(np.log(2 * np.pi * dy ** 2)
                         + ((y - model(theta, x)) / dy) ** 2)

What about priors? Remember your prior only depends on the model parameters, but be careful about what kind of prior you are specifying for each parameter. Do we need to properly normalize the probabilities?

In [ ]:
def ln_prior(theta):
    # flat prior: log(1) = 0
    return 0

Now we can define a function that evaluates the log-posterior probability, which is just the sum of the log-prior and log-likelihood:

In [ ]:
def ln_posterior(theta, x, y, dy):
    return ln_prior(theta) + ln_likelihood(theta, x, y, dy)

Now write a function to actually run a Metropolis-Hastings MCMC sampler. Ford (2005) includes a great step-by-step walkthrough of the Metropolis-Hastings algorithm, and we'll base our code on that. Fill-in the steps mentioned in the comments below:

In [ ]:
def run_mcmc(ln_posterior, nsteps, ndim, theta0, stepsize, args=()):
    """
    Run a Markov Chain Monte Carlo
    
    Parameters
    ----------
    ln_posterior: callable
        our function to compute the posterior
    nsteps: int
        the number of steps in the chain
    theta0: list
        the starting guess for theta
    stepsize: float
        a parameter controlling the size of the random step
        e.g. it could be the width of the Gaussian distribution
    args: tuple (optional)
        additional arguments passed to ln_posterior
    """
    # Create the array of size (nsteps, ndims) to hold the chain
    # Initialize the first row of this with theta0
    chain = np.zeros((nsteps, ndim))
    chain[0] = theta0
    
    # Create the array of size nsteps to hold the log-likelihoods for each point
    # Initialize the first entry of this with the log likelihood at theta0
    log_likes = np.zeros(nsteps)
    log_likes[0] = ln_posterior(chain[0], *args)
    
    # Loop for nsteps
    for i in range(1, nsteps):
        # Randomly draw a new theta from the proposal distribution.
        # for example, you can do a normally-distributed step by utilizing
        # the np.random.randn() function
        theta_new = chain[i - 1] + stepsize * np.random.randn(ndim)
        
        # Calculate the probability for the new state
        log_like_new = ln_likelihood(theta_new, *args)
        
        # Compare it to the probability of the old state
        # Using the acceptance probability function
        # (remember that you've computed the log probability, not the probability!)
        log_p_accept = log_like_new - log_likes[i - 1]
        
        # Chose a random number r between 0 and 1 to compare with p_accept
        r = np.random.rand()
        
        # If p_accept>1 or p_accept>r, accept the step
        # Else, do not accept the step
        if log_p_accept > np.log(r):
            chain[i] = theta_new
            log_likes[i] = log_like_new
        else:
            chain[i] = chain[i - 1]
            log_likes[i] = log_likes[i - 1]
            
    return chain

Now run the MCMC code on the data provided.

In [ ]:
chain = run_mcmc(ln_posterior, 10000, 2, [0, 1], 0.1, (x, y, dy))

Plot the position of the walker as a function of step number for each of the parameters. Are the chains converged? 

In [ ]:
fig, ax = plt.subplots(2)
ax[0].plot(chain[:, 0])
ax[1].plot(chain[:, 1]);

In [ ]:
# Now that we've burned-in, let's get a fresh chain
chain = run_mcmc(ln_posterior, 20000, 2, chain[-1], 0.1, (x, y, dy))

In [ ]:
fig, ax = plt.subplots(2)
ax[0].plot(chain[:, 0])
ax[1].plot(chain[:, 1]);

Make histograms of the samples for each parameter. Should you include all of the samples? 

In [ ]:
fig, ax = plt.subplots(2)
ax[0].hist(chain[:, 0], alpha=0.5)
ax[1].hist(chain[:, 1], alpha=0.5);

It's also sometimes useful to view a two-dimensional histogram:

In [ ]:
plt.hist2d(chain[:, 0], chain[:, 1], bins=30,
           cmap='Blues')
plt.xlabel('intercept')
plt.ylabel('slope')
plt.grid(False);

Report to us your constraints on the model parameters.
This is the number for the abstract – the challenge is to figure out how to accurately summarize a multi-dimensional posterior (which is **the result** in Bayesianism) with a few numbers (which is what readers want to see as they skim the arXiv).

What numbers should you use?

In [ ]:
theta_best = chain.mean(0)
theta_std = chain.std(0)

print("true intercept:", theta_true[0])
print("true slope:", theta_true[1])
print()
print("intercept = {0:.1f} +/- {1:.1f}".format(theta_best[0], theta_std[0]))
print("slope = {0:.2f} +/- {1:.2f}".format(theta_best[1], theta_std[1]))